In [ ]:
# motivated by 
# https://twitter.com/MariosRichards/status/1258726923318018048

In [1]:
dataset_name = "W19_comb"
df_list = [ "BES_Panel" ]

In [2]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

%matplotlib inline

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re

sns.set();
sns.set_palette("colorblind")

from IPython.display import display, display_html, HTML
from IPython.core.debugger import set_trace
# plt.rcParams["axes.grid"] = False

import Jupyter_module_loader
from utility import *
import gaussian_kde

import warnings
warnings.filterwarnings('ignore')

import holoviews as hv
from holoviews import opts

encoding = "ISO-8859-1"

# you should clone this git to a subdirectory called 'BES_analysis_code' (in some directory - I call it BES_analysis - doesn't matter though)
# %matplotlib inline
(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()

global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric
data_subfolder = BES_data_folder + dataset_name + os.sep

(manifest, dataset_filename, dataset_description, dataset_citation,
 dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)

for df in df_list:
    if df=="BES_Panel":
        globals()[df]  = pd.read_pickle(data_subfolder + dataset_filename.replace('.dta','.zip'),compression='zip')
    else:
        globals()[df]  = pd.read_pickle(data_subfolder + df + '.zip',compression='zip' )
        globals()[df].replace(-1,np.nan,inplace=True)
  
(var_type, cat_dictionary, new_old_col_names, old_new_col_names) = get_small_files(data_subfolder, encoding)

# get full set of inferred "cross wave" auth-lib/left-right values and ages
# pan_dataset_allr_values = pd.read_csv(BES_small_data_files + "pan_dataset_allr_values"+".csv")
# pan_dataset_ages = pd.read_pickle(BES_small_data_files + "pan_dataset_ages"+".zip", compression='zip')

Toggle code

var_type (7911, 14)


In [3]:
BESW19 = BES_Panel.copy()

In [6]:
BESW16 = BES_Panel.copy()

In [7]:
search(BESW16,"profile_past_vote_2010")

profile_past_vote_2010    63248
dtype: int64

In [8]:
search(BESW19,"profile_past_vote_2010")

profile_past_vote_2010    65581
dtype: int64

In [9]:
memory_use(locs = locals().items())

                        BESW19: 799.9MiB
                     BES_Panel: 695.0MiB
                        BESW16: 695.0MiB
                      var_type:   5.2MiB
                cat_dictionary: 288.1KiB
             new_old_col_names: 288.1KiB
             old_new_col_names: 144.1KiB
             BES_file_manifest:  58.5KiB
                           _i2:   2.5KiB
                           _i5:   2.5KiB


In [10]:
BESW19.shape,BESW16.shape

((97158, 7911), (92587, 6516))

In [13]:
BESW16 = BESW16.set_index("id").sort_index()
BESW19 = BESW19.set_index("id").sort_index()

In [15]:
BESW16["profile_past_vote_2010"].value_counts()

Conservative                                18121
Labour                                      14335
Liberal Democrat                            12307
I would not vote                             9655
Scottish National Party (SNP)                2290
Don't know                                   2018
United Kingdom Independence Party (UKIP)     1873
Green Party                                   993
Other                                         562
Plaid Cymru                                   550
British National Party (BNP)                  544
Name: profile_past_vote_2010, dtype: int64

In [16]:
BESW19["profile_past_vote_2010"].value_counts()

Conservative                                17934
Labour                                      16643
Liberal Democrat                            12853
Scottish National Party (SNP)                6158
I would not vote                             5001
United Kingdom Independence Party (UKIP)     1220
Brexit Party                                 1126
Don't know                                    952
British National Party (BNP)                  868
Plaid Cymru                                   829
10.0                                          741
Green Party                                   686
Other                                         371
Change UK- The Independent Group              199
Name: profile_past_vote_2010, dtype: int64

In [263]:
(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()

global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric

var_dict = {}

for dataset_name in BES_file_manifest[BES_file_manifest["Base_or_Strings"]=="Base"]["Name"]:
    
    data_subfolder = BES_data_folder + dataset_name + os.sep

    (manifest, dataset_filename, dataset_description, dataset_citation,
     dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)    
    try:
        df = pd.read_pickle(data_subfolder + dataset_filename.replace('.dta','.zip'),compression='zip')
        var_dict[dataset_name] = df.columns
    except:
        try:
            df = pd.read_stata(data_subfolder + dataset_filename).set_index("id").sort_index()
            var_dict[dataset_name] = df.columns
            df.to_pickle(data_subfolder + dataset_filename.replace('.dta','.zip'),compression='zip')
        except:
            var_dict[dataset_name] = "dta not present either"


In [264]:
[x for x in var_dict.keys() if isinstance(var_dict[x],str)]

[]

In [265]:
# ['W13_only', 'W11_only', 'W10_only', 'W9_only', 'W7_only']
# need updating!
# Also updated: W15_only, W14_only,W12_only, ... all the way down, but 'only' files ... only



# bug W5_only appears twice

In [266]:
[x for x in var_dict.keys() if not isinstance(var_dict[x],str) and "profile_past_vote_2010" in var_dict[x]]

['W19_only',
 'W19_comb',
 'W18_only',
 'W17_only',
 'W16_only',
 'W16_comb',
 'W15_only',
 'W15_comb',
 'W14_comb',
 'W13_comb',
 'W11_only',
 'W10_comb',
 'W10_only',
 'W9_comb',
 'W9_only',
 'W8_only',
 'W8_comb',
 'W7_only',
 'W7_comb',
 'W6_only',
 'W6_comb',
 'W5_comb',
 'W5_only',
 'W4_only',
 'W4_comb',
 'W3_only',
 'W3_comb',
 'W2_only',
 'W1_only']

In [267]:
[x for x in var_dict.keys() if not isinstance(var_dict[x],str) and "profile_past_vote_2005" in var_dict[x]]

# W12_only!

['W19_only',
 'W19_comb',
 'W18_only',
 'W17_only',
 'W16_only',
 'W16_comb',
 'W15_only',
 'W15_comb',
 'W14_comb',
 'W13_comb',
 'W12_only',
 'W11_only',
 'W10_comb',
 'W10_only',
 'W9_comb',
 'W9_only',
 'W8_only',
 'W8_comb',
 'W7_only',
 'W7_comb',
 'W6_only',
 'W6_comb',
 'W5_comb',
 'W5_only',
 'W4_only',
 'W4_comb',
 'W3_only',
 'W3_comb',
 'W2_only',
 'W1_only']

In [268]:
[x for x in var_dict["W12_only"] if "profile" in x]
# W12_only really missing profile_past_vote_2010!

['profile_newspaper2',
 'profile_gross_household',
 'profile_socgrade',
 'profile_work_stat',
 'profile_house_tenure',
 'profile_religion',
 'profile_education_age',
 'profile_lea',
 'profile_oslaua',
 'profile_gross_personal',
 'profile_household_children',
 'profile_past_vote_2005',
 'profile_past_vote_2017',
 'profile_turnout_2017',
 'profile_past_vote_2015',
 'profile_turnout_2015',
 'profile_eurefvote',
 'profile_eurefturnout',
 'profile_scotref_turnout',
 'profile_scotref_vote']

In [269]:
present_in_datasets = [x for x in var_dict.keys() if not isinstance(var_dict[x],str) and "profile_past_vote_2010" in var_dict[x]]
present_in_datasets.remove("W19_comb")
present_in_datasets.insert(0,"W19_comb")
present_in_datasets
# need W19_comb first to set all indices!

# Assumes no indices are lost ...

['W19_comb',
 'W19_only',
 'W18_only',
 'W17_only',
 'W16_only',
 'W16_comb',
 'W15_only',
 'W15_comb',
 'W14_comb',
 'W13_comb',
 'W11_only',
 'W10_comb',
 'W10_only',
 'W9_comb',
 'W9_only',
 'W8_only',
 'W8_comb',
 'W7_only',
 'W7_comb',
 'W6_only',
 'W6_comb',
 'W5_comb',
 'W5_only',
 'W4_only',
 'W4_comb',
 'W3_only',
 'W3_comb',
 'W2_only',
 'W1_only']

In [270]:
present_in_datasets = [x for x in present_in_datasets if "only" in x]
present_in_datasets.insert(0,"W19_comb")
present_in_datasets

['W19_comb',
 'W19_only',
 'W18_only',
 'W17_only',
 'W16_only',
 'W15_only',
 'W11_only',
 'W10_only',
 'W9_only',
 'W8_only',
 'W7_only',
 'W6_only',
 'W5_only',
 'W4_only',
 'W3_only',
 'W2_only',
 'W1_only']

In [271]:
# df = pd.DataFrame()
# temp = BESW16[["profile_past_vote_2005","profile_past_vote_2005"]]
# temp.columns = ["W16_"+x for x in temp.columns]
# df = pd.concat([df,temp],axis=1)
# temp = BESW19[["profile_past_vote_2005","profile_past_vote_2005"]]
# temp.columns = ["W19_"+x for x in temp.columns]
# df = pd.concat([df,temp],axis=1)

# df

In [272]:
%%time
(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()

global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric

df = pd.DataFrame()
var_list = ["profile_past_vote_2005","profile_past_vote_2010"]

for dataset_name in present_in_datasets:
    
    data_subfolder = BES_data_folder + dataset_name + os.sep

    (manifest, dataset_filename, dataset_description, dataset_citation,
     dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)    
    BES_Panel = pd.read_pickle(data_subfolder + dataset_filename.replace('.dta','.zip'),compression='zip')
    if "id" in BES_Panel.columns:
        temp = BES_Panel[var_list+["id"]].set_index("id").sort_index()
    else:
        temp = BES_Panel[var_list]
    temp.columns = [dataset_name+"_"+x for x in temp.columns]
    df = pd.concat([df,temp],axis=1)
    
df.index = df.index.astype('int')
# note - no extra rows over W19    

Wall time: 34.1 s


In [273]:
# vote2005_categories = list( df[[x for x in df.columns if "profile_past_vote_2005"]].stack().value_counts().index )

In [274]:
# vote2010_categories = list( df[[x for x in df.columns if "profile_past_vote_2010"]].stack().value_counts().index )

In [275]:
replace_dict = {'I would not vote':'Did not vote',
                'Conservative Party':'Conservative',
                'Labour Party':'Labour',
                'Liberal Democrats':'Liberal Democrat',
                'Scottish National Party (SNP)':'Scottish National Party',
                'Some other party':'Other',
               }
ppv2005 = [x for x in df.columns if "profile_past_vote_2005" in x]
df.loc[:,ppv2005] = df.loc[:,ppv2005].replace(replace_dict)
vote2005_categories = list( df[ppv2005].stack().value_counts().index )
vote2005_categories

['Conservative',
 'Labour',
 'Liberal Democrat',
 'Did not vote',
 'Scottish National Party',
 "Don't know",
 'An independent candidate',
 'United Kingdom Independence Party (UKIP)',
 'Green Party',
 'Plaid Cymru',
 'British National Party (BNP)',
 'Other',
 10.0,
 'Brexit Party',
 'Change UK- The Independent Group']

In [276]:
replace_dict = {'I would not vote':'Did not vote',
                'Conservative Party':'Conservative',
                'Labour Party':'Labour',
                'Liberal Democrats':'Liberal Democrat',
                'Scottish National Party (SNP)':'Scottish National Party',
                'Some other party':'Other',
               }
ppv2010 = [x for x in df.columns if "profile_past_vote_2010" in x]
df.loc[:,ppv2010] = df.loc[:,ppv2010].replace(replace_dict)
vote2010_categories = list( df[ppv2010].stack().value_counts().index )
vote2010_categories

['Conservative',
 'Labour',
 'Liberal Democrat',
 'Did not vote',
 'Scottish National Party',
 'United Kingdom Independence Party (UKIP)',
 "Don't know",
 'British National Party (BNP)',
 'Plaid Cymru',
 'Green Party',
 'Brexit Party',
 10.0,
 'Other',
 'Change UK- The Independent Group']

In [277]:
%%time
number_of_unique_responses_per_row_2005 = df[ppv2005].apply(lambda x: len((x.dropna()).unique()),axis=1)
number_of_unique_responses_per_row_2005.value_counts()

Wall time: 44.9 s


1    50268
0    40149
2     6741
dtype: int64

In [278]:
%%time
number_of_unique_responses_per_row_2010 = df[ppv2010].apply(lambda x: len((x.dropna()).unique()),axis=1)
number_of_unique_responses_per_row_2010.value_counts()

Wall time: 44.8 s


1    56419
0    31577
2     9162
dtype: int64

In [329]:
# df[ppv2005].loc[  number_of_unique_responses_per_row_2005>=3 ].dropna(how='all',axis=1)

In [280]:
discrepancy_pairs_2005 = df[ppv2005].loc[  number_of_unique_responses_per_row_2005==2 ].dropna(how='all',axis=1).apply(lambda x: "|".join([str(x) for x in x.dropna().unique()]),axis=1)

In [281]:
discrepancy_pairs_2010 = df[ppv2010].loc[  number_of_unique_responses_per_row_2010==2 ].dropna(how='all',axis=1).apply(lambda x: "|".join([str(x) for x in x.dropna().unique()]),axis=1)

In [282]:
discrepancy_pairs_2010.value_counts()

Labour|Conservative                                     2946
Liberal Democrat|Labour                                 1680
Conservative|Did not vote                               1652
Scottish National Party|Liberal Democrat                1634
Brexit Party|Don't know                                  454
10.0|United Kingdom Independence Party (UKIP)            322
British National Party (BNP)|Green Party                 185
Plaid Cymru|Scottish National Party                      113
Change UK- The Independent Group|Other                    82
Green Party|British National Party (BNP)                  65
United Kingdom Independence Party (UKIP)|Plaid Cymru      29
dtype: int64

In [283]:
discrepancy_pairs_2005.value_counts()

Labour|Conservative                                     1896
Liberal Democrat|Labour                                 1593
Conservative|Did not vote                               1287
Scottish National Party|Liberal Democrat                 904
An independent candidate|Don't know                      662
British National Party (BNP)|Green Party                 130
10.0|United Kingdom Independence Party (UKIP)            118
Plaid Cymru|Scottish National Party                       57
Brexit Party|Other                                        47
Green Party|British National Party (BNP)                  25
United Kingdom Independence Party (UKIP)|Plaid Cymru      20
Change UK- The Independent Group|Other                     2
dtype: int64

In [284]:
df["discrepancy_pairs_2005"] = discrepancy_pairs_2005
df["discrepancy_pairs_2010"] = discrepancy_pairs_2010

In [285]:
# df[df["discrepancy_pairs_2005"]=="10.0|United Kingdom Independence Party (UKIP)"]["discr_2005_change_dataset"].value_counts()

In [286]:
df["discr_2005_change_dataset"] = np.nan
# discrep_pair = "Other|Veritas"
for discrep_pair in discrepancy_pairs_2005.value_counts().index:
    pair_first_part = discrep_pair.split("|")[0]
    if pair_first_part=="10.0":
        pair_first_part=10.0
    temp_ser = (df.loc[discrepancy_pairs_2005[discrepancy_pairs_2005==discrep_pair].index,ppv2005]==pair_first_part).replace(False,np.nan).cumsum(axis=1).replace(True,1.0).idxmax(axis=1)
    df.loc[temp_ser.index,"discr_2005_change_dataset"] = temp_ser
df.groupby("discr_2005_change_dataset")["discrepancy_pairs_2005"].value_counts()    

discr_2005_change_dataset        discrepancy_pairs_2005                              
W10_only_profile_past_vote_2005  Labour|Conservative                                     454
                                 Conservative|Did not vote                               409
                                 Liberal Democrat|Labour                                 398
                                 Scottish National Party|Liberal Democrat                240
                                 An independent candidate|Don't know                     210
                                                                                        ... 
W8_only_profile_past_vote_2005   Plaid Cymru|Scottish National Party                      25
                                 Green Party|British National Party (BNP)                 11
                                 Brexit Party|Other                                        8
                                 United Kingdom Independence Party (UKIP)|Pla

In [287]:
df["discr_2010_change_dataset"] = np.nan
# discrep_pair = "Other|Veritas"
for discrep_pair in discrepancy_pairs_2010.value_counts().index:
    pair_first_part = discrep_pair.split("|")[0]
    if pair_first_part=="10.0":
        pair_first_part=10.0    
    temp_ser = (df.loc[discrepancy_pairs_2010[discrepancy_pairs_2010==discrep_pair].index,ppv2010]==pair_first_part).replace(False,np.nan).cumsum(axis=1).replace(True,1.0).idxmax(axis=1)
  
    df.loc[temp_ser.index,"discr_2010_change_dataset"] = temp_ser
df.groupby("discr_2010_change_dataset")["discrepancy_pairs_2010"].value_counts()

discr_2010_change_dataset        discrepancy_pairs_2010                              
W10_only_profile_past_vote_2010  Labour|Conservative                                     674
                                 Conservative|Did not vote                               479
                                 Scottish National Party|Liberal Democrat                410
                                 Liberal Democrat|Labour                                 377
                                 Brexit Party|Don't know                                 123
                                                                                        ... 
W8_only_profile_past_vote_2010   British National Party (BNP)|Green Party                 29
                                 Plaid Cymru|Scottish National Party                      27
                                 Green Party|British National Party (BNP)                 20
                                 Change UK- The Independent Group|Other      

In [288]:
df["discr_2010_change_dataset"].value_counts().sort_index()

W10_only_profile_past_vote_2010    2259
W11_only_profile_past_vote_2010     891
W15_only_profile_past_vote_2010    1633
W17_only_profile_past_vote_2010     132
W18_only_profile_past_vote_2010      62
W19_comb_profile_past_vote_2010      81
W19_only_profile_past_vote_2010       6
W7_only_profile_past_vote_2010     2062
W8_only_profile_past_vote_2010     2036
Name: discr_2010_change_dataset, dtype: int64

In [289]:
df["discr_2005_change_dataset"].value_counts().sort_index()

W10_only_profile_past_vote_2005    1830
W11_only_profile_past_vote_2005     725
W15_only_profile_past_vote_2005     339
W17_only_profile_past_vote_2005      34
W18_only_profile_past_vote_2005      12
W19_comb_profile_past_vote_2005      24
W4_only_profile_past_vote_2005        2
W7_only_profile_past_vote_2005     1835
W8_only_profile_past_vote_2005     1940
Name: discr_2005_change_dataset, dtype: int64

In [290]:
df["discrepancy_pairs_2005"].value_counts()

Labour|Conservative                                     1896
Liberal Democrat|Labour                                 1593
Conservative|Did not vote                               1287
Scottish National Party|Liberal Democrat                 904
An independent candidate|Don't know                      662
British National Party (BNP)|Green Party                 130
10.0|United Kingdom Independence Party (UKIP)            118
Plaid Cymru|Scottish National Party                       57
Brexit Party|Other                                        47
Green Party|British National Party (BNP)                  25
United Kingdom Independence Party (UKIP)|Plaid Cymru      20
Change UK- The Independent Group|Other                     2
Name: discrepancy_pairs_2005, dtype: int64

In [291]:
df["discrepancy_pairs_2010"].value_counts()

Labour|Conservative                                     2946
Liberal Democrat|Labour                                 1680
Conservative|Did not vote                               1652
Scottish National Party|Liberal Democrat                1634
Brexit Party|Don't know                                  454
10.0|United Kingdom Independence Party (UKIP)            322
British National Party (BNP)|Green Party                 185
Plaid Cymru|Scottish National Party                      113
Change UK- The Independent Group|Other                    82
Green Party|British National Party (BNP)                  65
United Kingdom Independence Party (UKIP)|Plaid Cymru      29
Name: discrepancy_pairs_2010, dtype: int64

In [292]:
df["W7_only_profile_past_vote_2005"].value_counts()

Conservative                                8919
Labour                                      7793
Liberal Democrat                            4585
Did not vote                                2657
Scottish National Party                     1474
Don't know                                   883
An independent candidate                     430
United Kingdom Independence Party (UKIP)     408
Green Party                                  348
Plaid Cymru                                  303
Other                                        199
British National Party (BNP)                 198
10.0                                          83
Brexit Party                                  32
Change UK- The Independent Group               1
Name: W7_only_profile_past_vote_2005, dtype: int64

In [308]:
df

,W19_comb_profile_past_vote_2005,W19_comb_profile_past_vote_2010,W19_only_profile_past_vote_2005,W19_only_profile_past_vote_2010,W18_only_profile_past_vote_2005,W18_only_profile_past_vote_2010,W17_only_profile_past_vote_2005,W17_only_profile_past_vote_2010,W16_only_profile_past_vote_2005,W16_only_profile_past_vote_2010,...,W3_only_profile_past_vote_2005,W3_only_profile_past_vote_2010,W2_only_profile_past_vote_2005,W2_only_profile_past_vote_2010,W1_only_profile_past_vote_2005,W1_only_profile_past_vote_2010,discrepancy_pairs_2005,discrepancy_pairs_2010,discr_2005_change_dataset,discr_2010_change_dataset
1,Labour,United Kingdom Independence Party (UKIP),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Labour,United Kingdom Independence Party (UKIP),Labour,United Kingdom Independence Party (UKIP),NaN,NaN,NaN,NaN
2,Conservative,Conservative,Conservative,Conservative,Conservative,Conservative,Conservative,Conservative,NaN,NaN,...,Conservative,Conservative,NaN,NaN,Conservative,Conservative,NaN,NaN,NaN,NaN
3,Labour,Liberal Democrat,Labour,Liberal Democrat,NaN,NaN,NaN,NaN,Labour,Liberal Democrat,...,Labour,Liberal Democrat,Labour,Liberal Democrat,Labour,Liberal Democrat,NaN,NaN,NaN,NaN
4,Conservative,Conservative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Conservative,Conservative,Conservative,Conservative,NaN,NaN,NaN,NaN
5,Did not vote,Liberal Democrat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Did not vote,Liberal Democrat,Did not vote,Liberal Democrat,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [293]:
DNV2005 = (df["W7_only_profile_past_vote_2005"]=="Did not vote")
DNV2010 = (df["W7_only_profile_past_vote_2010"]=="Did not vote")

In [310]:
BES_Panel = BES_Panel.set_index("id").sort_index()

In [311]:
first_wave_joined_panel = BES_Panel[search(BES_Panel,"wave\d+").index].replace(False,np.nan).fillna(method='ffill',axis=1).replace(np.nan,False).astype('bool')
first_wave_joined_panel.columns = [int(x.replace("wave","")) for x in first_wave_joined_panel.columns]

In [313]:
for wave in range(1,20):
    mask  = list(set(first_wave_joined_panel[first_wave_joined_panel[wave]].index).union()  )
    print( wave,
          DNV2010.loc[first_wave_joined_panel[first_wave_joined_panel[wave]].index].sum(),
          DNV2010.loc[first_wave_joined_panel[~first_wave_joined_panel[wave]].index].sum(),
          DNV2005.loc[first_wave_joined_panel[first_wave_joined_panel[wave]].index].sum(),
          DNV2005.loc[first_wave_joined_panel[~first_wave_joined_panel[wave]].index].sum(),
         )

1 1661 714 1990 667
2 1836 539 2180 477
3 2028 347 2298 359
4 2374 1 2657 0
5 2375 0 2657 0
6 2375 0 2657 0
7 2375 0 2657 0
8 2375 0 2657 0
9 2375 0 2657 0
10 2375 0 2657 0
11 2375 0 2657 0
12 2375 0 2657 0
13 2375 0 2657 0
14 2375 0 2657 0
15 2375 0 2657 0
16 2375 0 2657 0
17 2375 0 2657 0
18 2375 0 2657 0
19 2375 0 2657 0


In [309]:
df.shape, BES_Panel.shape

((97158, 38), (97158, 7911))

In [300]:
temp_df = df.copy()

In [303]:
df = temp_df.copy()

In [322]:
var_counts = pd.DataFrame.from_dict( {k:v for k,v in var_dict.items() if k in present_in_datasets} , orient='index').stack().value_counts()

In [327]:
var_counts[var_counts==17].index

Index(['housing', 'profile_past_vote_2005', 'profile_turnout_2017',
       'profile_past_vote_2017', 'euRefLA', 'profile_past_vote_2010',
       'profile_newspaper', 'profile_scotref_turnout', 'profile_turnout_2015',
       'profile_gross_household', 'profile_lea', 'profile_eurefvote',
       'profile_past_vote_2015', 'profile_gross_personal', 'disability',
       'profile_household_children', 'gender', 'profile_scotref_vote',
       'profile_education_age', 'marital', 'ageGroup'],
      dtype='object')

In [388]:
var_counts[var_counts==4].index

Index(['natSecurityGrn', 'profile_work_type', 'responsibleDebtEU',
       'responsibleImmigLibDems', 'happyEULeave', 'scotRefID4', 'respdate',
       'responsibleDebtDontKnow', 'approveEU', 'responsibleCostLiveLab',
       ...
       'responsibleEducationWelshGovt', 'responsibleImmigConservatives',
       'responsibleImmigLabour', 'partyContactBrexit_4',
       'responsibleNHSScottishGovt', 'partyContactBrexit_7', 'refAreGood',
       'responsibleCostLiveNone', 'beliefTradeoff',
       'responsibleEducationScotGovt'],
      dtype='object', length=116)

In [ ]:
# profile_eurefturnout, profile_ethnicity, profile_religion, countryOfBirth, profile_past_vote_2015

In [373]:
var_counts.value_counts().sort_index()

1     8383
2      913
3      127
4      116
5       97
6       55
7       57
8       22
9       39
10      34
11      21
12      13
13      69
14      25
15      25
16      18
17      21
dtype: int64

In [330]:
present_in_datasets

['W19_comb',
 'W19_only',
 'W18_only',
 'W17_only',
 'W16_only',
 'W15_only',
 'W11_only',
 'W10_only',
 'W9_only',
 'W8_only',
 'W7_only',
 'W6_only',
 'W5_only',
 'W4_only',
 'W3_only',
 'W2_only',
 'W1_only']

In [333]:
%%time
(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()

global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric

df = pd.DataFrame()
var_list = list(var_counts[var_counts==17].index)

for dataset_name in present_in_datasets:
    
    data_subfolder = BES_data_folder + dataset_name + os.sep

    (manifest, dataset_filename, dataset_description, dataset_citation,
     dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)    
    BES_Panel = pd.read_pickle(data_subfolder + dataset_filename.replace('.dta','.zip'),compression='zip')
    if "id" in BES_Panel.columns:
        temp = BES_Panel[var_list+["id"]].set_index("id").sort_index()
    else:
        temp = BES_Panel[var_list]
    temp.columns = [dataset_name+"_"+x for x in temp.columns]
    df = pd.concat([df,temp],axis=1)
    
df.index = df.index.astype('int')
# note - no extra rows over W19    

Wall time: 51.7 s


In [334]:
%%time

number_of_unique_responses_per_row = {}

for focal_var in var_list:
    var_cols = [x for x in df.columns if focal_var in x]

    number_of_unique_responses_per_row[focal_var] = df[var_cols].apply(lambda x: len((x.dropna()).unique()),axis=1)
    number_of_unique_responses_per_row[focal_var].value_counts()

Wall time: 20min 46s


In [338]:
for key in number_of_unique_responses_per_row.keys():
    print(key)
    display(number_of_unique_responses_per_row[key].value_counts())

housing


0    65301
1    31857
dtype: int64

profile_past_vote_2005


1    50268
0    40149
2     6741
dtype: int64

profile_turnout_2017


...

1    39216
0    23432
2    21590
3    12920
dtype: int64

marital


1    64090
0    33068
dtype: int64

ageGroup


1    66930
0    30228
dtype: int64

In [ ]:
# discrepancy_pairs_2005 = df[ppv2005].loc[  number_of_unique_responses_per_row_2005==2 ].dropna(how='all',axis=1).apply(lambda x: "|".join([str(x) for x in x.dropna().unique()]),axis=1)

In [339]:
%%time

for focal_var in var_list:
    var_cols = [x for x in df.columns if focal_var in x]
    df["nuniq_"+focal_var] = number_of_unique_responses_per_row[focal_var]
    df["discrep_"+focal_var] = df[var_cols].loc[  df["nuniq_"+focal_var]>=2 ].dropna(how='all',axis=1).apply(lambda x: "|".join([str(x) for x in x.dropna().unique()]),axis=1)
    

Wall time: 1min 35s


In [342]:
df

,W19_comb_housing,W19_comb_profile_past_vote_2005,W19_comb_profile_turnout_2017,W19_comb_profile_past_vote_2017,W19_comb_euRefLA,W19_comb_profile_past_vote_2010,W19_comb_profile_newspaper,W19_comb_profile_scotref_turnout,W19_comb_profile_turnout_2015,W19_comb_profile_gross_household,...,nuniq_gender,discrep_gender,nuniq_profile_scotref_vote,discrep_profile_scotref_vote,nuniq_profile_education_age,discrep_profile_education_age,nuniq_marital,discrep_marital,nuniq_ageGroup,discrep_ageGroup
1,Own the leasehold/freehold outright,Labour,NaN,NaN,E06000050,United Kingdom Independence Party (UKIP),The Mirror / Daily Record,NaN,"Yes, voted","Â£10,000 to Â£14,999 per year",...,1,NaN,0,NaN,1,NaN,1,NaN,1,NaN
2,Own the leasehold/freehold outright,Conservative,"Yes, voted",Conservative,E07000215,Conservative,The Daily Telegraph,NaN,"Yes, voted","Â£50,000 to Â£59,999 per year",...,1,NaN,0,NaN,3,20+|d|Foreman or Supervisor of Other Workers (...,1,NaN,1,NaN
3,Own the leasehold/freehold outright,Labour,"Yes, voted",Labour,E07000136,Liberal Democrat,The Mirror / Daily Record,NaN,"Yes, voted","Â£5,000 to Â£9,999 per year",...,1,NaN,0,NaN,2,20+|d,1,NaN,1,NaN
4,Buying leasehold/freehold on a mortgage,Conservative,NaN,NaN,NaN,Conservative,The Sun,NaN,NaN,"Â£25,000 to Â£29,999 per year",...,1,NaN,0,NaN,1,NaN,1,NaN,1,NaN
5,Own the leasehold/freehold outright,I would not vote,NaN,NaN,NaN,Liberal Democrat,The Daily Telegraph,NaN,NaN,"Â£25,000 to Â£29,999 per year",...,1,NaN,0,NaN,1,NaN,1,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98447,NaN,NaN,"No, did not vote",NaN,NaN,NaN,NaN,NaN,"No, did not vote",NaN,...,1,NaN,0,NaN,0,NaN,0,NaN,0,NaN
98448,NaN,NaN,"No, did not vote",NaN,NaN,NaN,NaN,NaN,"No, did not vote",NaN,...,1,NaN,0,NaN,0,NaN,0,NaN,0,NaN
98449,NaN,NaN,"Yes, voted",Don't know,NaN,NaN,NaN,NaN,"Yes, voted",NaN,...,1,NaN,0,NaN,1,NaN,0,NaN,0,NaN
98450,NaN,NaN,"No, did not vote",NaN,NaN,NaN,NaN,NaN,Don't know,NaN,...,1,NaN,0,NaN,0,NaN,0,NaN,0,NaN


In [347]:
df[[x for x in df.columns if "discrep" in x]].dropna(how='all',axis=1)

,discrep_profile_past_vote_2005,discrep_profile_past_vote_2010,discrep_profile_newspaper,discrep_profile_turnout_2015,discrep_profile_gross_household,discrep_profile_gross_personal,discrep_disability,discrep_profile_household_children,discrep_gender,discrep_profile_education_age
1,NaN,NaN,The Mirror / Daily Record|None,NaN,"Â£10,000 to Â£14,999 per year|Â£25,000 to Â£29...",NaN,"Yes, limited a little|No",NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,"Â£50,000 to Â£59,999 per year|£60,000 to £69,9...","Â£50,000 to Â£59,999 per year|£10,000 to £14,9...",NaN,NaN,NaN,20+|d|Foreman or Supervisor of Other Workers (...
3,NaN,NaN,NaN,NaN,"Â£5,000 to Â£9,999 per year|£5,000 to £9,999 p...","Â£5,000 to Â£9,999 per year|£5,000 to £9,999 p...",NaN,NaN,NaN,20+|d
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
98447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [349]:
df["discrep_gender"].value_counts()

Female|Male    7
Male|Female    1
Name: discrep_gender, dtype: int64

In [352]:
BES_Panel.loc[df[df["discrep_gender"].notnull()].index,]

Int64Index([30929, 35852, 38388, 39244, 76846, 77218, 77875, 89977], dtype='int64')

In [353]:
search(BES_Panel,"sexuality")

Series([], dtype: float64)

In [360]:
df[df[[x for x in df.columns if "discrep" in x]].dropna(how='all',axis=1).columns[3]].value_counts()

No, did not vote|Yes, voted    33
Yes, voted|No, did not vote    21
Yes, voted|Don't know          20
No, did not vote|Don't know    19
Don't know|Yes, voted          13
Don't know|No, did not vote    11
Name: discrep_profile_turnout_2015, dtype: int64

In [368]:
df[df[[x for x in df.columns if "discrep" in x]].dropna(how='all',axis=1).columns[8]].value_counts().head(20)

Female|Male    7
Male|Female    1
Name: discrep_gender, dtype: int64

In [391]:
%%time
(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()

global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric

df = pd.DataFrame()
var_list = ["profile_eurefturnout", "profile_ethnicity", "profile_religion", "countryOfBirth", "profile_past_vote_2015"]

for dataset_name in present_in_datasets:
    
    data_subfolder = BES_data_folder + dataset_name + os.sep

    (manifest, dataset_filename, dataset_description, dataset_citation,
     dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)    
    BES_Panel = pd.read_pickle(data_subfolder + dataset_filename.replace('.dta','.zip'),compression='zip')
    pres_var_list = [x for x in var_list if x in BES_Panel.columns]
    if "id" in BES_Panel.columns:
        temp = BES_Panel[pres_var_list+["id"]].set_index("id").sort_index()
    else:
        temp = BES_Panel[pres_var_list]
    temp.columns = [dataset_name+"_"+x for x in temp.columns]
    df = pd.concat([df,temp],axis=1)
    
df.index = df.index.astype('int')
# note - no extra rows over W19    

Wall time: 30.4 s


In [392]:
%%time

number_of_unique_responses_per_row = {}

for focal_var in var_list:
    var_cols = [x for x in df.columns if focal_var in x]

    number_of_unique_responses_per_row[focal_var] = df[var_cols].apply(lambda x: len((x.dropna()).unique()),axis=1)
    number_of_unique_responses_per_row[focal_var].value_counts()

Wall time: 4min 56s


In [393]:
%%time

for focal_var in var_list:
    var_cols = [x for x in df.columns if focal_var in x]
    df["nuniq_"+focal_var] = number_of_unique_responses_per_row[focal_var]
    df["discrep_"+focal_var] = df[var_cols].loc[  df["nuniq_"+focal_var]>=2 ].dropna(how='all',axis=1).apply(lambda x: "|".join([str(x) for x in x.dropna().unique()]),axis=1)
    

Wall time: 6.39 s


In [397]:
df[[x for x in df.columns if "discrep" in x]].dropna(how='all',axis=1).columns

Index(['discrep_profile_ethnicity', 'discrep_profile_religion',
       'discrep_countryOfBirth'],
      dtype='object')

In [406]:
df[['discrep_profile_ethnicity', 'discrep_profile_religion',
       'discrep_countryOfBirth'][2]].value_counts().head(5)

Other: Rest of world|Other: Commonwealth member country    27
Other: Commonwealth member country|Other: Rest of world    27
England|Other: Rest of world                               25
Prefer not to answer|England                               15
England|Wales                                              10
Name: discrep_countryOfBirth, dtype: int64

In [404]:
df[['discrep_profile_ethnicity', 'discrep_profile_religion',
       'discrep_countryOfBirth'][1]].value_counts().head(5)

Yes - Church of England/Anglican/Episcopal|No, I do not regard myself as belonging to any particular religion.    1615
No, I do not regard myself as belonging to any particular religion.|Yes - Church of England/Anglican/Episcopal    1434
No, I do not regard myself as belonging to any particular religion.|Prefer not to say                              705
Yes - Other|No, I do not regard myself as belonging to any particular religion.                                    300
Yes - Roman Catholic|No, I do not regard myself as belonging to any particular religion.                           294
Name: discrep_profile_religion, dtype: int64

In [407]:
df[['discrep_profile_ethnicity', 'discrep_profile_religion',
       'discrep_countryOfBirth'][0]].value_counts().head(5)

Any other white background|White British        6
White British|Any other white background        4
Any other mixed background|White British        3
Prefer not to say|White British                 2
Any other Asian background|Prefer not to say    1
Name: discrep_profile_ethnicity, dtype: int64

In [ ]:
##############################################################################

# comparing W19_comb
# BES_Panel,BES_Panelv02



In [3]:
BES_Panelv02 = pd.read_pickle("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES_Panelv02",compression='zip')

In [4]:
BES_Panel.shape, BES_Panelv02.shape

((97158, 7911), (97006, 8080))

In [72]:
gone_vars = list(set(BES_Panel.columns) - set(BES_Panelv02.columns))
old_stubs = {re.match("(.*?W\d+)",x).groups()[0]:x for x in gone_vars if re.match(".*W\d+W\d+",x)}
gone_vars

['likeBorisW19',
 'likeBorisW13',
 'profile_work_industryW14',
 'lr4W10W11W12',
 'al4W10W11W12',
 'profile_education_ageW13',
 'fatherVoteW17',
 'pastvote_eup_2019W19',
 'countryOfBirth',
 'likeBorisW9',
 'profile_oslaua',
 'euRefpastVoteW17',
 'profile_lea',
 'educationW13',
 'profile_education_ageW19',
 'motherVoteW17',
 'profile_mpnameW2',
 'lr_scaleW7W8W9',
 'profile_household_sizeW18',
 'al1W7W8W9',
 'profile_eurefturnout',
 'headHouseholdPastW4',
 'educationW18',
 'voted2017W19',
 'lr2W7W8W9',
 'profile_gross_household',
 'educationW15',
 'sexualityW17',
 'currentUnionMemberW2_W12_W14W15',
 'gayEqualityW1W2W3W4W5',
 'educationW6',
 'motherNumEmployeesW13',
 'educationW12',
 'scotReferendumRetroW15',
 'educationW1_W6',
 'lr3W10W11W12',
 'profile_work_organisationW18',
 'motherNumEmployeesW10',
 'profile_house_tenureW10',
 'profile_household_childrenW18',
 'profile_turnout_2017',
 'al2W10W11W12',
 'profile_marital_statW14',
 'educationW3',
 'lr5W10W11W12',
 'al5W7W8W9',
 'marital',
 'profile_scotRefW18',
 'profile_gross_householdW16',
 'al3W1W2W3W4W5',
 'voted2015',
 'countryOfBirthW17',
 'lr1W10W11W12',
 'disability',
 'profile_past_vote_2015',
 'sickElderlyInHouseW1W2W3W4W5',
 'profile_gross_householdW15',
 'profile_socgradeW14',
 'lr4W1W2W3W4W5',
 'scotReferendumVoteW11',
 'profile_ethnicity2W17',
 'headHouseholdPastW9',
 'profile_house_tenureW14',
 'schoolKidsInHouseW1W2W3W4W5',
 'profile_work_statW14',
 'edlevelW8W9W10',
 'profile_scotRefW19',
 'scotReferendumVoteW15',
 'likeBorisW8',
 'al3W10W11W12',
 'fatherNumEmployeesW11',
 'euRefpastVoteW14',
 'educationW2',
 'everUnionMemberW17',
 'profile_gross_householdW12',
 'profile_religionW17',
 'al5W10W11W12',
 'profile_gross_personalW18',
 'profile_work_typeW7W10',
 'everUnionMemberW16',
 'profile_socgradeW12',
 'lr1W1W2W3W4W5',
 'headHouseholdPastW10',
 'profile_work_organisationW19',
 'polKnowBercowW1W2W3',
 'profile_religion',
 'al2W7W8W9',
 'ethnicity2W15',
 'profile_religionW14',
 'profile_religionW10',
 'profile_turnout_2015',
 'profile_socialgrade_cieW18',
 'noDependentsInHouseW1W2W3W4W5',
 'al3W7W8W9',
 'al4W7W8W9',
 'profile_marital_statW18',
 'al2W1W2W3W4W5',
 'profile_past_vote_2010',
 'profile_work_typeW2_W4',
 'profile_marital_statW16',
 'blackEqualityW1W2W3W4W5',
 'profile_house_tenureW12',
 'profile_gross_personalW7',
 'likeBorisW17',
 'profile_gross_householdW18',
 'profile_ethnicity2W19',
 'profile_religionW13',
 'polKnowMilibandW1W2W3',
 'edlevelW7',
 'educationW7',
 'polKnowPutinW2W3W4',
 'euRefLA',
 'profile_house_tenureW17',
 'edlevelW14',
 'profile_gross_householdW13',
 'motherNumEmployees',
 'profile_household_sizeW19',
 'profile_work_industryW19',
 'edlevelW1_W6',
 'al_scaleW7W8W9',
 'educationW9',
 'profile_work_statW7',
 'profile_newspaperW11',
 'profile_household_sizeW10',
 'profile_house_tenure',
 'profile_work_statW13',
 'lr3W1W2W3W4W5',
 'motherNumEmployeesW11',
 'polKnowOsborneW1W2W3',
 'polKnowNetanyahuW2W3W4W7W9',
 'profile_work_responsibilityW18',
 'profile_mpnameW3',
 'profile_religionW11',
 'educationW14',
 'likeBorisW7',
 'lr_scaleW10W11W12',
 'headHouseholdPastW11',
 'profile_socialgrade_cieW17',
 'currentUnionMemberW17',
 'profile_work_statW19',
 'al1W10W11W12',
 'profile_household_children',
 'profile_work_responsibilityW19',
 'fatherNumEmployeesW10',
 'ethnicity2W16',
 'profile_gross_householdW17',
 'profile_gross_householdW7',
 'profile_work_industryW17',
 'profile_scotref_vote',
 'polKnowMerkelW2W3W4',
 'profile_work_typeW17',
 'profile_socgradeW7',
 'profile_work_typeW14',
 'subjectHECurrentW1W2W3',
 'profile_work_statW18',
 'headHouseholdPastW3',
 'profile_scotRefW17',
 'profile_work_responsibilityW17',
 'everUnionMemberW2_W12_W14W15',
 'profile_newspaper2W12',
 'educationW19',
 'profile_ethnicity',
 'profile_work_statW11',
 'profile_socgradeW15',
 'lr2W10W11W12',
 'fatherNumEmployeesW13',
 'profile_household_childrenW19',
 'profile_religionW16',
 'motherVoteW14W15W16',
 'al5W1W2W3W4W5',
 'profile_work_statW12',
 'profile_work

In [66]:
new_vars= list(set(BES_Panelv02.columns) - set(BES_Panel.columns))
# [x for x in new_vars if 
new_stubs = {x:re.match("(.*?W\d+)",x).groups()[0] for x in new_vars if re.match(".*W\d+_",x)}
# [x for x in new_vars if not re.match("(age|pano|pcon|country|gor)|(p_)|(.*W\d+_W\d+)|(likeJohnson|enviroGrowth|oslaua|parentNumEmployees)",x)]

# new columns
# profile style columns for waves that weren't included before (age/pano/pcon/country/gor/)
# enviroGrowth has reappeared???
# variables where the formatting for multiple waves has been made more consistent
# likeBoris -> likeJohnson

In [70]:
# al2W10_W12
old_to_new_dict = {}
for new_key in new_stubs.keys():
    if new_stubs[new_key] in old_stubs.keys():
        old_to_new_dict[old_stubs[new_stubs[new_key]]] = new_key
    else:
        print("doesn't appear to have older analogue",new_key)


doesn't appear to have older analogue motherVoteW3W14_W17
doesn't appear to have older analogue fatherVoteW3W14_W17


In [101]:
for new_var in new_vars:
    if "likeJohnson" in new_var:
        old_to_new_dict[new_var.replace("likeJohnson","likeBoris")] = new_var
    elif "parentNumEmployees" in new_var:
        old_to_new_dict[new_var.replace("parentNumEmployees","fatherNumEmployees")] = new_var
        old_to_new_dict[new_var.replace("parentNumEmployees","motherNumEmployees")] = new_var
#     else:
#         print("doesn't appear to have older analogue",new_key)

In [133]:
old_to_new_dict["profile_sexualityW10"] = "xsexualityW10"

old_to_new_dict["profile_scotref_vote"] = "p_vote_scot_ref"
old_to_new_dict["profile_scotref_turnout"] = "p_turnout_scot_ref"

old_to_new_dict["newspaperReadW15"] = "p_paper_readW15"
old_to_new_dict["profile_newspaper2W12"] = "p_paper_readW12"
old_to_new_dict["profiles_newspaper2W16"] = "p_paper_readW16"
old_to_new_dict["profiles_newspaper2W17"] = "p_paper_readW17"

old_to_new_dict["ethnicity2W15"] = "p_ethnicityW15"
old_to_new_dict["ethnicity2W16"] = "p_ethnicityW16"
old_to_new_dict["profile_ethnicity2W17"] = "p_ethnicityW17"
old_to_new_dict["profile_ethnicity2W18"] = "p_ethnicityW18"
old_to_new_dict["profile_ethnicity2W19"] = "p_ethnicityW19"



In [136]:

drop_vars = ["work_type","work_organisation","work_responsibility","work_industry","profile_lea","profile_oslaua","profile_mpname"]

for x in [x for x in gone_vars if "profile" in x and x not in old_to_new_dict.keys()]:
    
    if x.replace("profile","p") in new_vars:
        old_to_new_dict[x] = x.replace("profile","p")
    elif x.replace("profile_household_size","p_hh_size") in new_vars:
        old_to_new_dict[x] = x.replace("profile_household_size","p_hh_size")
    elif x.replace("profile_marital_stat","p_marital") in new_vars:
        old_to_new_dict[x] = x.replace("profile_marital_stat","p_marital")
    elif x.replace("profile_house_tenure","p_housing") in new_vars:
        old_to_new_dict[x] = x.replace("profile_house_tenure","p_housing")
    elif x.replace("profile_socialgrade_cie","p_socgrade") in new_vars:
        old_to_new_dict[x] = x.replace("profile_socialgrade_cie","p_socgrade")
    elif x.replace("profile_newspaper","p_paper_read") in new_vars:
        old_to_new_dict[x] = x.replace("profile_newspaper","p_paper_read")
        
        
        
        
        
    elif not np.any([y in x for y in drop_vars]) and re.match("\d",x[-1]):
        print(x, " not in new vars")
        
        
        
# many of the "amalgamated waveless" variables have disappeared
# work_type|organisation|responsibility|industry
# profile_lea,profile_leaW17,profile_oslaua
# profile_mpnameW2,profile_mpnameW3


# profile_scotRefW17,profile_scotRefW18,profile_scotRefW19
# profile_household_childrenW18,profile_household_childrenW19
# profile_work_statW1_W10


profile_household_childrenW18  not in new vars
profile_scotRefW18  not in new vars
profile_scotRefW19  not in new vars
profile_scotRefW17  not in new vars
profile_household_childrenW19  not in new vars
profile_work_statW1_W10  not in new vars


In [137]:
[x for x in new_vars if "p_" in x and x not in old_to_new_dict.values()]

# disability and sexuality largely new

# 



['p_sexualityW13',
 'p_sexualityW9',
 'p_religionW6',
 'p_disabilityW13',
 'p_paper_readW4',
 'p_sexualityW17',
 'p_gross_personalW17',
 'p_edlevelUniW8',
 'p_religionW1',
 'p_gross_householdW1',
 'p_edlevelW16',
 'p_edlevelW5',
 'p_maritalW6',
 'p_parentW10',
 'p_sexualityW15',
 'p_parentW15',
 'p_edlevelW9',
 'p_gross_personalW1',
 'p_maritalW12',
 'p_hh_sizeW17',
 'p_hh_sizeW11',
 'p_parentW11',
 'p_gross_personalW2',
 'p_parentW9',
 'p_educationW11',
 'p_edlevelW4',
 'p_education_ageW8',
 'p_ethnicityW6',
 'p_disabilityW4',
 'p_ethnicityW1',
 'p_maritalW3',
 'p_sexualityW18',
 'p_hh_sizeW14',
 'p_socgradeW11',
 'p_turnout_2010',
 'p_disabilityW14',
 'p_gross_personalW13',
 'p_parentW16',
 'p_sexualityW2',
 'p_disabilityW2',
 'p_work_statW9',
 'p_past_vote_2019',
 'p_religionW2',
 'p_parentW8',
 'p_turnout_2005',
 'p_educationW15',
 'p_educationW3',
 'p_gross_personalW4',
 'p_scot_list_vote_2011',
 'p_edlevelUniW19',
 'p_housingW3',
 'p_paper_readW6',
 'p_maritalW7',
 'p_edlevelW7',
 'p_disabilityW12',
 'p_ethnicityW13',
 'p_edlevelW8',
 'p_educationW8',
 'p_hh_sizeW9',
 'p_sexualityW8',
 'p_edlevelUniW9',
 'p_housingW5',
 'p_education_ageW12',
 'p_gross_personalW16',
 'p_disabilityW16',
 'p_paper_readW14',
 'p_educationW17',
 'p_educationW19',
 'p_edlevelUniW17',
 'p_educationW7',
 'p_maritalW9',
 'p_edlevelW11',
 'p_gross_householdW3',
 'p_hh_sizeW13',
 'p_disabilityW18',
 'p_disabilityW15',
 'p_edlevelUniW7',
 'p_maritalW1',
 'p_paper_readW8',
 'p_scot_const_vote_2016',
 'p_sexualityW3',
 'p_maritalW4',
 'p_educationW14',
 'p_hh_sizeW16',
 'p_sexualityW6',
 'p_education_ageW1',
 'p_maritalW8',
 'p_educationW16',
 'p_scot_const_vote_2011',
 'p_educationW6',
 'p_ethnicityW4',
 'p_hh_sizeW3',
 'p_sexualityW11',
 'p_ethnicityW10',
 'p_sexualityW4',
 'p_disabilityW1',
 'p_ethnicityW3',
 'p_hh_sizeW6',
 'p_work_statW15',
 'p_paper_readW3',
 'p_work_statW2',
 'p_welsh_const_vote_2016',
 'p_gross_householdW4',
 'p_socgradeW2',
 'p_housingW2',
 'p_educationW9',
 'p_hh_sizeW12',
 'p_sexualityW7',
 'p_paper_readW19',
 'p_hh_sizeW2',
 'p_gross_householdW9',
 'p_parentW13',
 'p_parentW17',
 'p_sexualityW19',
 'p_housingW19',
 'p_disabilityW11',
 'p_education_ageW15',
 'p_edlevelUniW10',
 'p_religionW7',
 'p_educationW10',
 'p_education_ageW17',
 'p_parentW14',
 'p_sexualityW10',
 'p_disabilityW17',
 'p_edlevelW14',
 'p_disabilityW3',
 'p_welsh_list_vote_2016',
 'p_hh_sizeW4',
 'p_socgradeW9',
 'p_gross_householdW5',
 'p_paper_readW9',
 'p_work_statW6',
 'p_hh_sizeW8',
 'p_religionW18',
 'p_gross_personalW3',
 'p_gross_personalW15',
 'p_educationW18',
 'p_work_statW1',
 'p_education_ageW3',
 'p_edlevelW2',
 'p_work_statW10',
 'p_edlevelUniW18',
 'p_scot_list_vote_2016',
 'p_paper_readW7',
 'p_educationW12',
 'p_ethnicityW14',
 'p_ethnicityW9',
 'p_work_statW5',
 'p_sexualityW14',
 'p_edlevelW3',
 'p_education_ageW10',
 'p_work_statW8',
 'p_religionW4',
 'p_gross_personalW11',
 'p_housingW7',
 'p_housingW8',
 'p_maritalW5',
 'p_edlevelUniW12',
 'p_gross_personalW10',
 'p_edlevelW15',
 'p_edlevelW17',
 'p_education_ageW16',
 'p_ethnicityW11',
 'p_educationW4',
 'p_hh_sizeW5',
 'p_gross_personalW8',
 'p_housingW18',
 'p_ethnicityW2',
 'p_edlevelUniW14',
 'p_edlevelW1',
 'p_educationW5',
 'p_edlevelW6',
 'p_sexualityW16',
 'p_gross_householdW8',
 'p_edlevelUniW11',
 'p_parentW12',
 'p_education_ageW14',
 'p_paper_readW1',
 'p_disabilityW10',
 'p_maritalW19',
 'p_gross_householdW6',
 'p_socgradeW6',
 'p_turnout_2019',
 'p_maritalW2',
 'p_religionW3',
 'p_gross_personalW12',
 'p_gross_personalW14',
 'p_housingW9',
 'p_edlevelW10',
 'p_hh_sizeW15',
 'p_educationW2',
 'p_gross_householdW2',
 'p_disabilityW7',
 'p_ethnicityW5',
 'p_education_ageW11',
 'p_religionW5',
 'p_education_ageW5',
 'p_gross_personalW6',
 'p_edlevelW19',
 'p_disabilityW5',
 'p_gross_personalW9',
 'p_paper_readW5',
 'p_sexualityW12',
 'p_education_ageW6',
 'p_edlevelUniW15',
 'p_work_statW3',
 'p_housingW4',
 'p_ethnicityW8',
 'p_housingW1',
 'p_parentW19',
 'p_ethnicity

In [130]:
[x for x in new_vars if x not in old_to_new_dict.values()]

['p_sexualityW13',
 'p_sexualityW9',
 'p_religionW6',
 'p_disabilityW13',
 'oslauaW9',
 'p_paper_readW4',
 'p_sexualityW17',
 'p_gross_personalW17',
 'p_edlevelUniW8',
 'p_religionW1',
 'p_gross_householdW1',
 'p_edlevelW16',
 'p_edlevelW5',
 'p_maritalW6',
 'p_parentW10',
 'p_sexualityW15',
 'countryW18',
 'p_parentW15',
 'p_edlevelW9',
 'p_gross_personalW1',
 'p_maritalW12',
 'oslauaW19',
 'p_hh_sizeW17',
 'oslauaW14',
 'p_hh_sizeW11',
 'gorW20',
 'p_parentW11',
 'p_gross_personalW2',
 'p_parentW9',
 'p_educationW11',
 'p_edlevelW4',
 'p_education_ageW8',
 'p_ethnicityW6',
 'p_disabilityW4',
 'oslauaW2',
 'p_ethnicityW1',
 'p_maritalW3',
 'p_sexualityW18',
 'p_hh_sizeW14',
 'p_socgradeW11',
 'p_turnout_2010',
 'p_disabilityW14',
 'p_gross_personalW13',
 'p_ethnicityW19',
 'p_parentW16',
 'p_sexualityW2',
 'oslauaW16',
 'p_disabilityW2',
 'p_work_statW9',
 'oslauaW15',
 'p_past_vote_2019',
 'p_religionW2',
 'enviroGrowthW17',
 'p_parentW8',
 'p_turnout_2005',
 'ageW3',
 'enviroGrowthW7',
 'p_educationW15',
 'p_educationW3',
 'p_gross_personalW4',
 'p_scot_list_vote_2011',
 'oslauaW3',
 'p_edlevelUniW19',
 'p_ethnicityW17',
 'gorW18',
 'p_housingW3',
 'p_paper_readW6',
 'p_maritalW7',
 'p_edlevelW7',
 'p_disabilityW12',
 'xsexualityW10',
 'p_ethnicityW13',
 'p_edlevelW8',
 'p_educationW8',
 'p_hh_sizeW9',
 'p_sexualityW8',
 'oslauaW10',
 'p_edlevelUniW9',
 'p_housingW5',
 'p_education_ageW12',
 'p_gross_personalW16',
 'p_disabilityW16',
 'p_paper_readW14',
 'p_ethnicityW18',
 'p_educationW17',
 'p_educationW19',
 'p_edlevelUniW17',
 'p_educationW7',
 'p_maritalW9',
 'p_edlevelW11',
 'p_gross_householdW3',
 'oslauaW11',
 'p_hh_sizeW13',
 'p_disabilityW18',
 'p_disabilityW15',
 'motherVoteW3W14_W17',
 'p_edlevelUniW7',
 'p_maritalW1',
 'p_paper_readW8',
 'p_scot_const_vote_2016',
 'p_sexualityW3',
 'p_maritalW4',
 'p_educationW14',
 'p_hh_sizeW16',
 'p_sexualityW6',
 'p_education_ageW1',
 'p_maritalW8',
 'p_educationW16',
 'p_scot_const_vote_2011',
 'p_educationW6',
 'p_turnout_scot_ref',
 'p_ethnicityW4',
 'p_hh_sizeW3',
 'p_sexualityW11',
 'p_ethnicityW10',
 'p_sexualityW4',
 'p_disabilityW1',
 'p_ethnicityW3',
 'p_hh_sizeW6',
 'p_ethnicityW15',
 'p_work_statW15',
 'p_paper_readW3',
 'p_work_statW2',
 'countryW4',
 'p_welsh_const_vote_2016',
 'p_gross_householdW4',
 'panoW18',
 'p_socgradeW2',
 'p_housingW2',
 'p_educationW9',
 'p_hh_sizeW12',
 'p_sexualityW7',
 'oslauaW1',
 'p_paper_readW19',
 'p_hh_sizeW2',
 'oslauaW18',
 'p_gross_householdW9',
 'p_parentW13',
 'p_parentW17',
 'p_sexualityW19',
 'p_housingW19',
 'oslauaW13',
 'p_disabilityW11',
 'p_education_ageW15',
 'p_edlevelUniW10',
 'panoW4',
 'p_religionW7',
 'p_educationW10',
 'p_education_ageW17',
 'oslauaW17',
 'p_parentW14',
 'oslauaW5',
 'p_disabilityW17',
 'p_edlevelW14',
 'p_disabilityW3',
 'p_welsh_list_vote_2016',
 'p_hh_sizeW4',
 'p_socgradeW9',
 'p_gross_householdW5',
 'oslauaW8',
 'p_paper_readW9',
 'oslauaW6',
 'p_work_statW6',
 'p_hh_sizeW8',
 'p_religionW18',
 'p_gross_personalW3',
 'p_gross_personalW15',
 'p_educationW18',
 'p_work_statW1',
 'p_education_ageW3',
 'p_edlevelW2',
 'p_work_statW10',
 'p_edlevelUniW18',
 'p_scot_list_vote_2016',
 'p_paper_readW7',
 'enviroGrowthW16',
 'p_educationW12',
 'p_ethnicityW14',
 'p_ethnicityW9',
 'p_work_statW5',
 'p_sexualityW14',
 'p_edlevelW3',
 'p_education_ageW10',
 'p_work_statW8',
 'p_religionW4',
 'genElecTurnoutRetroW7',
 'p_gross_personalW11',
 'p_housingW7',
 'pconW18',
 'p_housingW8',
 'fatherVoteW3W14_W17',
 'p_maritalW5',
 'p_edlevelUniW12',
 'p_gross_personalW10',
 'pconW4',
 'p_edlevelW15',
 'p_edlevelW17',
 'p_education_ageW16',
 'p_ethnicityW11',
 'p_educationW4',
 'p_hh_sizeW5',
 'p_gross_personalW8',
 'p_housingW18',
 'p_ethnicityW2',
 'p_edlevelUniW14',
 'p_edlevelW1',
 'p_educationW5',
 'p_edlevelW6',
 'p_sexualityW16',
 'p_gross_householdW8',
 'p_edlevelUniW11',
 'p_parentW12',
 'p_education_ageW14',
 'p_paper_readW1',
 'p_disabilityW10',
 'p_maritalW19',
 'p_gross_householdW6',
 'p_socgrade

In [128]:
BES_Panel["profile_house_tenureW14"].value_counts()

Own - outright                                                                          11908
Own - with a mortgage                                                                    8136
Rent - from a private landlord                                                           3826
Rent - from a housing association                                                        1941
Rent - from my local authority                                                           1618
Neither - I live rent-free with my parents, family or friends                            1075
Neither - I live with my parents, family or friends but pay some rent to them            1036
Other                                                                                     492
Own (part-own) - through shared ownership scheme (i.e. pay part mortgage, part rent)      206
Name: profile_house_tenureW14, dtype: int64

In [121]:
BES_Panelv02["p_housingW14"].value_counts()

Own â outright                                                                          11938
Own â with a mortgage                                                                    8165
Rent â from a private landlord                                                           3893
Rent â from a housing association                                                        1950
Rent â from my local authority                                                           1643
Neither â I live rent-free with my parents, family or friends                            1098
Neither â I live with my parents, family or friends but pay some rent to them            1063
Other                                                                                       482
Own (part-own) â through shared ownership scheme (i.e. pay part mortgage, part rent)      214
Name: p_housingW14, dtype: int64

In [13]:
BES_Panelv02[['xsexualityW10', 'genElecTurnoutRetroW7', 'voted2015newwordingW15']]

,xsexualityW10,genElecTurnoutRetroW7,voted2015newwordingW15
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,Heterosexual,"Yes, voted",NaN
4,NaN,NaN,NaN
...,...,...,...
97001,NaN,NaN,NaN
97002,NaN,NaN,NaN
97003,NaN,"Yes, voted",NaN
97004,Heterosexual,Don't know,NaN


In [38]:
BES_Panelv02['voted2015newwordingW15'].value_counts()

Yes, voted          1374
No, did not vote     519
Don't know           181
Name: voted2015newwordingW15, dtype: int64

In [37]:
BES_Panelv02['genElecTurnoutRetroW7'].value_counts()

Yes, voted          27307
No, did not vote     2791
Don't know            797
Name: genElecTurnoutRetroW7, dtype: int64

In [39]:
(BES_Panelv02['voted2015newwordingW15']==BES_Panelv02['genElecTurnoutRetroW7']).sum()
# no overlap ... assume the w15 is supplemental?

0

In [14]:
BES_Panelv02["xsexualityW10"].value_counts()

Heterosexual         26801
Prefer not to say     1144
Gay or lesbian        1104
Bisexual               911
Other                  277
Name: xsexualityW10, dtype: int64

In [30]:
BES_Panelv02["p_sexualityW10"].value_counts()

Heterosexual         26805
Prefer not to say     1139
Gay or lesbian        1105
Bisexual               912
Other                  275
Name: p_sexualityW10, dtype: int64

In [31]:
BES_Panelv02["xsexualityW10"].notnull().sum(),BES_Panelv02["p_sexualityW10"].notnull().sum()

(30237, 30236)

In [32]:
set(BES_Panelv02["xsexualityW10"][BES_Panelv02["xsexualityW10"].notnull()].index)\
    -set(BES_Panelv02["p_sexualityW10"][BES_Panelv02["p_sexualityW10"].notnull()].index)
    

{67489}

In [33]:
(BES_Panelv02["xsexualityW10"] != BES_Panelv02["p_sexualityW10"]).sum()

23

In [35]:
BES_Panelv02[["xsexualityW10","p_sexualityW10"]][(BES_Panelv02["xsexualityW10"] != BES_Panelv02["p_sexualityW10"])]



,xsexualityW10,p_sexualityW10
2311,Prefer not to say,Heterosexual
5382,Prefer not to say,Heterosexual
5841,Prefer not to say,Heterosexual
7928,Other,Prefer not to say
8071,Heterosexual,Gay or lesbian
16664,Prefer not to say,Bisexual
16812,Heterosexual,Prefer not to say
19758,Heterosexual,Prefer not to say
20053,Prefer not to say,Heterosexual
30290,Prefer not to say,Heterosexual


In [42]:
BES_Panelv02["p_past_vote_2017"].value_counts()

Labour                                      23742
Conservative                                23741
Liberal Democrat                             5258
Scottish National Party (SNP)                2523
United Kingdom Independence Party (UKIP)     1498
Green Party                                  1289
Don't know                                   1204
Other                                         726
Plaid Cymru                                   395
British National Party (BNP)                    1
Name: p_past_vote_2017, dtype: int64

In [109]:
BES_Panel["sexualityW17"].value_counts()

Yes    29976
No      1317
5.0     1316
3.0     1057
4.0      359
Name: sexualityW17, dtype: int64

In [111]:
BES_Panelv02["p_sexualityW17"].value_counts()

Heterosexual         29993
Gay or lesbian        1321
Prefer not to say     1319
Bisexual              1062
Other                  360
Name: p_sexualityW17, dtype: int64

In [112]:
BES_Panel["sexualityW14"].value_counts()

No                   7014
Yes                   539
Prefer not to say     248
Name: sexualityW14, dtype: int64

In [114]:
BES_Panelv02["p_sexualityW14"].value_counts()

Heterosexual         27422
Prefer not to say     1304
Gay or lesbian        1082
Bisexual               851
Other                  284
Name: p_sexualityW14, dtype: int64

In [131]:
search(BES_Panel,"ethnicity")

discussantEthnicity1W2    24933
discussantEthnicity2W2    18238
discussantEthnicity3W2    13210
ethnicity2W15             30842
ethnicity2W16             37958
profile_ethnicityW7       30833
profile_ethnicity2W17     34364
profile_ethnicity2W18     37825
profile_ethnicity2W19     32177
profile_ethnicity         70290
dtype: int64